In [1]:
import numpy as np
from collections import deque
from datetime import datetime,timedelta
from typing import Dict

# Mock classes to replace Django models
class User:
    def __init__(self, username, balance):
        self.username = username
        self.balance = balance
        self.portfolio = Portfolio()

class Portfolio:
    def __init__(self):
        self.stocks = []

    def add(self, stock):
        self.stocks.append(stock)

    def remove(self, stock):
        self.stocks.remove(stock)

class Stock:
    def __init__(self, ticker, price, low_price, high_price,volatility,liquidity):
        self.ticker = ticker
        self.price = price
        self.low_price = low_price
        self.high_price = high_price
        self.volatility = volatility
        self.liquidity = liquidity

    def get_spread(self):

        return ((self.high_price-self.low_price)/self.price)*100

    def update_price(self, new_price):
      
        self.price = new_price
        StockHistory().record(self)

class StockHistory:
    _instance = None

    def __new__(cls):
        if cls._instance is None:
            cls._instance = super(StockHistory, cls).__new__(cls)
            cls._instance.history = []
        return cls._instance

    def record(self, stock):
        self.history.append({
            'timestamp': datetime.now().isoformat(),
            'ticker': stock.ticker,
            'price': stock.price,
            'low_price': stock.low_price,
            'high_price': stock.high_price,
            'volatility': stock.volatility
        })

    def get_history(self):
        return self.history

    def get_prices(self, ticker, time_period):
        # Assuming self.history is a list of dictionaries, each representing a recorded stock state
        filtered_prices = []
        for record in self.history:
            if record['ticker'] == ticker:
                # Assuming record['price'] is the price at the recorded time
                filtered_prices.append(record['price'])

        # Return prices based on time_period
        return filtered_prices[-time_period:]

class TransactionHistory:
    transactions = []

    @classmethod
    def create(cls, portfolio, asset, transaction_type, amount, price):
        cls.transactions.append({
            'portfolio': portfolio,
            'asset': asset,
            'transaction_type': transaction_type,
            'amount': amount,
            'price': price,
            'timestamp': datetime.now().isoformat()
        })


In [2]:
class BuySellQueue:
    def __init__(self, stock: Stock):
        self.stock = stock
        self.buy_queue = deque()
        self.sell_queue = deque()

    def add_to_buy_queue(self, user, asset, amount, price):
        self.buy_queue.append((user, asset, amount, price))

    def add_to_sell_queue(self, user, asset, amount, price):
        self.sell_queue.append((user, asset, amount, price))


    def process_queues(self):
        transactions = []
        while len(self.buy_queue) and len(self.sell_queue):
            buy_order = self.buy_queue.popleft()
            sell_order = self.sell_queue.popleft()
            if buy_order[3] >= sell_order[3]:  # Buy price >= Sell price
                matched_price = (buy_order[3] + sell_order[3]) / 2
                transactions.append(self.execute_transaction(buy_order, sell_order, matched_price))

        return transactions

    def execute_transaction(self, buy_order, sell_order, price):
        
        buyer, asset, amount, buy_price = buy_order
        seller, _, sell_amount, sell_price = sell_order
        
        if sell_amount >= amount:
            self.complete_transaction(buyer, seller, asset, amount, price)
            if sell_amount > amount:
                self.add_to_sell_queue(seller, asset, sell_amount - amount, sell_price)

        return {
            'buyer': buyer.username,
            'seller': seller.username,
            'asset': asset.ticker,
            'amount': amount,
            'price': price,
            'timestamp': datetime.now().isoformat()
        }

    def complete_transaction(self, buyer, seller, asset, amount, price):
        asset.update_price(price)
        seller.portfolio.remove(asset)
        buyer.portfolio.add(asset)
        seller.balance += price * amount
        buyer.balance -= price * amount
        self.log_transaction(buyer, asset, 'buy', amount, price)
        self.log_transaction(seller, asset, 'sell', amount, price)
        

    def log_transaction(self, user, asset, transaction_type, amount, price):
        TransactionHistory.create(
            portfolio=user.portfolio,
            asset=asset,
            transaction_type=transaction_type,
            amount=amount,
            price=price
        )



In [3]:
class Broker:
    def __init__(self, name):
        self.name = name
        self.k = 0.5  # A constant factor representing the market maker's risk tolerance and desired profit margin.
        self.c = 0.01  # A constant representing fixed costs or other market-specific adjustments.
        self.queues: Dict[str, BuySellQueue] = {}
        self.stock_history = StockHistory()

    def get_queue(self, stock: Stock):
        if stock.ticker not in self.queues:
            self.queues[stock.ticker] = BuySellQueue(stock)
        return self.queues[stock.ticker] 
  

    def adjust_client_price(self, best_bid, best_ask, spread, transaction_type):
        midprice = (best_bid + best_ask) / 2
        if transaction_type == "buy":
            ask_price = midprice + spread / 2
            return ask_price
        else:
            bid_price = midprice - spread / 2
            return bid_price

    def get_best_prices(self, asset: Stock):
        best_bid = asset.low_price
        best_ask = asset.high_price
        return best_bid, best_ask

    def add_to_buysell_queue(self, user, asset: Stock, amount, transaction_type):
        stock_queue = self.get_queue(asset)
        best_bid, best_ask = self.get_best_prices(asset)
    
        spread = 0.01 * amount

        if transaction_type == "buy":
            adjusted_price = self.adjust_client_price(best_bid, best_ask, spread, "buy")
            stock_queue.add_to_buy_queue(user, asset, amount, adjusted_price)
            if adjusted_price > best_ask:
                asset.high_price = adjusted_price
        else:
            adjusted_price = self.adjust_client_price(best_bid, best_ask, spread, "sell")
            stock_queue.add_to_sell_queue(user, asset, amount, adjusted_price)
            if adjusted_price < best_bid:
                asset.low_price = adjusted_price
       

    def process_queues(self):
        transactions = []
        for queue in self.queues.values():
            transactions.extend(queue.process_queues())
        return transactions


In [4]:
# Create mock users
alice = User("Alice", 10000)
george = User('George',10000)
bob = User("Bob", 10000)

# Create a stock
apple_stock = Stock("AAPL", 150, 150, 150,1,10)

bob.portfolio.add(apple_stock)
bob.portfolio.add(apple_stock)
bob.portfolio.add(apple_stock)
bob.portfolio.add(apple_stock)
bob.portfolio.add(apple_stock)
# Create a broker
broker = Broker("Market Maker")

# Add orders to the queues
broker.add_to_buysell_queue(alice, apple_stock, 10, "buy")
broker.add_to_buysell_queue(alice, apple_stock, 10, "buy")
broker.add_to_buysell_queue(alice, apple_stock, 10, "buy")
broker.add_to_buysell_queue(george,apple_stock,10,"buy")
broker.add_to_buysell_queue(george,apple_stock,10,"buy")
broker.process_queues()

broker.add_to_buysell_queue(bob, apple_stock, 10, "sell")
broker.add_to_buysell_queue(bob, apple_stock, 10, "sell")
broker.add_to_buysell_queue(bob, apple_stock, 10, "sell")
broker.add_to_buysell_queue(bob, apple_stock, 10, "sell")
broker.add_to_buysell_queue(bob, apple_stock, 10, "sell")
# Process the queues
transactions = broker.process_queues()
# print(apple_stock.low_price,apple_stock.price,apple_stock.high_price)
# Print out the transactions 
for transaction in transactions:
    print(transaction)

transactions = broker.process_queues()
for transaction in transactions:
    print(transaction)

{'buyer': 'Alice', 'seller': 'Bob', 'asset': 'AAPL', 'amount': 10, 'price': 150.02421875, 'timestamp': '2024-07-16T09:02:12.111055'}
{'buyer': 'Alice', 'seller': 'Bob', 'asset': 'AAPL', 'amount': 10, 'price': 150.036328125, 'timestamp': '2024-07-16T09:02:12.111055'}
{'buyer': 'Alice', 'seller': 'Bob', 'asset': 'AAPL', 'amount': 10, 'price': 150.04238281250002, 'timestamp': '2024-07-16T09:02:12.111055'}
{'buyer': 'George', 'seller': 'Bob', 'asset': 'AAPL', 'amount': 10, 'price': 150.04541015625, 'timestamp': '2024-07-16T09:02:12.111055'}
{'buyer': 'George', 'seller': 'Bob', 'asset': 'AAPL', 'amount': 10, 'price': 150.046923828125, 'timestamp': '2024-07-16T09:02:12.111055'}
